In [1]:
import spacy

In [2]:
with open('share') as data_file:
        sentences = data_file.readlines()

In [9]:
sentences[:15]

['After surviving accident, Manjrekar shares pic of shattered door\n',
 'Shuttler Ajay Jayaram decorates rangoli on Diwali, shares picture\n',
 "Dhoni shares 'PUBG fan' Kohli's old pic with toy gun on birthday\n",
 'Pitch invader shares selfie clicked by Cristiano Ronaldo for him\n',
 'Anything is possible: Kohli shares pic of physical transformation\n',
 "Krunal shares 'legendary' pictures of brother Hardik on birthday\n",
 'Jwala shares #MeToo story of mental harassment by badminton chief\n',
 "Juventus' shares fall 5% after rape allegations against Ronaldo\n",
 'Rohit, Dhawan share 210-run stand vs Pak, break 20-year-old record\n',
 'Two teams share top spot for the first time in FIFA rankings\n',
 "Zlatan shares 'God of Goals' poster made in honour of 500th goal\n",
 'Messi shares photos of him dropping his kids to school\n',
 'Murray shares image of bloodied nose after daughter hits him\n',
 'Dhawan shares video of improvised slip catching drill\n',
 "Novak shares 'naked' ice bath

In [100]:
SPACY_DEP_ROOT = "ROOT"
SPACY_DEP_NSUBJ = "nsubj"
SPACY_DEP_DOBJ = "dobj"
SPACY_DEP_PREP = "prep"
SPACY_DEP_PREP_OBJ = "pobj"
SPACY_DEP_CCOMP = "ccomp"
SPACY_TAG_PREP = "IN"

In [101]:
nlp = spacy.load('en_core_web_sm')

In [102]:
def printTree(sentence):
	doc = nlp(str(sentence))
	 
	for token in doc:
	    print("{5}: {0}/{1} <--{2}-- {3}/{4} > {6}".format(
	        token.text, token.tag_, token.dep_, token.head.text, token.head.tag_,token.i,token.head.i))
        
printTree(u"Uthappa shows pic of physical transformation ahead of IPL")
parse(u"Uthappa shows pic of physical transformation ahead of IPL")
print(getCompoundPhrase("Uthappa shows pic of physical transformation ahead of IPL", 5))
#printTree(u"After surviving accident, Manjrekar shows pic of shattered door")
#parse(u"After surviving accident, Manjrekar shows pic of shattered door")

0: Uthappa/NNP <--nsubj-- shows/VBZ > 1
1: shows/VBZ <--ROOT-- shows/VBZ > 1
2: pic/NN <--dobj-- shows/VBZ > 1
3: of/IN <--prep-- pic/NN > 2
4: physical/JJ <--amod-- transformation/NN > 5
5: transformation/NN <--pobj-- of/IN > 3
6: ahead/RB <--advmod-- shows/VBZ > 1
7: of/IN <--prep-- ahead/RB > 6
8: IPL/NN <--pobj-- of/IN > 7
Uthappa shows pic of physical transformation ahead of IPL who:  Uthappa type:  pic what:  physical transformation
physical transformation


In [103]:
def getCompoundPhrase(sentence, rootTokenId):
    doc = nlp(str(sentence))
    phrase = []
    for token in doc:
        if (token.tag_ != SPACY_TAG_PREP) and (token.head.i == rootTokenId or token.i == rootTokenId):
            phrase.append(token.text)
    reversed(phrase)
    return ' '.join(phrase)

In [104]:
def getHeadOfSentence(sentence):

    doc = nlp(str(sentence))
    head = None
    for token in doc:
        if token.dep_ == SPACY_DEP_ROOT:
            #print ("head",sentence, token.text)
            head = token.text
            
    return head    

In [105]:
def parse(sentence):
    doc = nlp(str(sentence))
    whoShared = None
    whoSharedId = None
    
    typeShared = None
    typeSharedId = None
    
    whatShared = None
    whatSharedId = None
    
    prep = None
    prepId = None
    
    sentenceHead = getHeadOfSentence(sentence)
    #print("sent head: ", sentenceHead)
    for token in doc:
        # if token.dep_ == SPACY_DEP_ROOT:
        #     print (sentence, token.text)
        if token.dep_ == SPACY_DEP_NSUBJ and token.head.text == sentenceHead:
            whoShared = token.text
            whoSharedId = token.i
        if (token.dep_ == SPACY_DEP_DOBJ or token.dep_ == SPACY_DEP_CCOMP) and token.head.text == sentenceHead :
            typeShared = token.text
            typeSharedId = token.i
            
    #Figuring out preposition for typeShared       
    for token in doc:
        if token.dep_ == SPACY_DEP_PREP and token.head.text == typeShared:
            prep = token.text
            prepId = token.i
            break
    if prep != None:
        for token in doc:
            if token.dep_ == SPACY_DEP_PREP_OBJ and token.head.i == prepId:
                whatShared = token.text
                whatSharedId = token.i
    #print(sentence, "who: ", whoShared, "type: ", typeShared, "what: ", whatShared)
    print(sentence, 
          "who: ", getCompoundPhrase(sentence, whoSharedId), 
          "type: ", getCompoundPhrase(sentence, typeSharedId), 
          "what: ", getCompoundPhrase(sentence, whatSharedId))

# with -> subj with someone ; attached to noun(video, pic)
# in, at -> place ; attached to subj
# of -> content ; attached to noun(video, pic)
# on -> timeframe

In [106]:
for sentence in sentences:
	sentence = sentence.replace('shares', 'shows')
	sentence = sentence.replace('share', 'show')
	#getHeadOfSentence(sentence)
	parse(sentence)

After surviving accident, Manjrekar shows pic of shattered door
 who:  Manjrekar type:  pic what:  shattered door 

Shuttler Ajay Jayaram decorates rangoli on Diwali, shows picture
 who:   type:  picture 
 what:  
Dhoni shows 'PUBG fan' Kohli's old pic with toy gun on birthday
 who:  Dhoni type:  fan Kohli old pic what:  toy gun
Pitch invader shows selfie clicked by Cristiano Ronaldo for him
 who:  Pitch invader type:  selfie clicked what:  
Anything is possible: Kohli shows pic of physical transformation
 who:  Kohli type:  pic what:  physical transformation 

Krunal shows 'legendary' pictures of brother Hardik on birthday
 who:   type:   what:  
Jwala shows #MeToo story of mental harassment by badminton chief
 who:  Jwala type:  # MeToo story what:  mental harassment
Juventus' shows fall 5% after rape allegations against Ronaldo
 who:  Juventus shows type:   what:  
Rohit, Dhawan show 210-run stand vs Pak, break 20-year-old record
 who:   type:   what:  
Two teams show top spot for t